# <center> Beyond 2D inventories: synoptic 3D landslide volume calculation from repeat LiDAR data - Workflow</center>

In [1]:
import os
import importlib
import numpy as np
import pandas as pd
import scripts.cloudcompare as cc
import scripts.os_functions as osf
import scripts.functions as fc

# Methods: 3D point cloud differencing to detect and measure landslides

## 3D Landslide mapping workflow
<hr size="1"></hr>

### Master files

<table><tr>
    <td> <img src="figures/Master_files.png" width=400 height=400/> </td>
</tr></table>

In [2]:
from scripts.Work_environment_and_settings import *

### Geomorphic change detection


<table><tr>
    <td> <img src="figures/Master_files_2.png" width=400 height=400/> </td>
</tr></table>

In [3]:
osf.open_folder(workspace+workflow_folder)
commande1 = 'CloudCompare -SILENT -C_EXPORT_FMT ASC -SEP COMMA  -ADD_HEADER -AUTO_SAVE off -NO_TIMESTAMP'

In [ ]:
# Compute 3D-M3C2
commande = cc.open_file(cc.open_file(cc.open_file(commande0,shift,workspace+Data_folder+filenames['pre_EQ_lidar']+'.laz'),shift,workspace+Data_folder+filenames['post_EQ_lidar']+'.laz'),shift,workspace+Data_folder+filenames['core_points']+'.laz')
cc.m3c2(commande,params_file_3Dm3c2,workspace,Data_folder,filenames,file_extension='.las')
os.rename(workspace+Data_folder+list(filenames.values())[0]+'_M3C2.las',workspace+workflow_folder+'3D_M3C2_temp.las')

In [5]:
# Extract nan from distance uncertainty scalar field 
fc.split_SF(path=workspace+workflow_folder,SF='distance uncertainty')

In [6]:
# Compute 3D-M3C2
filenames['core_points'] = '3D_M3C2_nan_temp'
commande = cc.open_file(cc.open_file(cc.open_file(commande1,shift,workspace+Data_folder+filenames['pre_EQ_lidar']+'.laz'),shift,workspace+Data_folder+filenames['post_EQ_lidar']+'.laz'),shift,workspace+workflow_folder+filenames['core_points']+'.asc')
cc.m3c2(commande,params_file_m3c2_nan,workspace,Data_folder,filenames,file_extension='.asc')
os.rename(workspace+Data_folder+list(filenames.values())[0]+'_M3C2.asc',workspace+workflow_folder+'3D_M3C2_nan_d(10)_temp.asc')

In [7]:
# Merge both the 3D M3C2 on NaN and the first 3D M3C2 
commande = cc.open_file(cc.open_file(commande0,shift,workspace+workflow_folder+'3D_M3C2_split_temp.asc'),shift,workspace+workflow_folder+'3D_M3C2_nan_d(10)_temp.asc')
cc.merge_clouds(commande)
os.rename(workspace+workflow_folder+'3D_M3C2_split_temp'+'_MERGED.las',workspace+workflow_folder+'3D_M3C2.las')
# Delete all temp files
osf.delete_file_extension(workspace+workflow_folder,'.asc')

In [19]:
# Extract significant changes (SC)
commande= cc.open_file(commande0,shift,workspace+workflow_folder+'3D_M3C2.las')
cc.filter_SF(commande,indexSF=2,min=1,max=2)
os.rename(workspace+workflow_folder+'3D_M3C2_FILTERED_[1_2].las',workspace+workflow_folder+'Significant_change.las')

<div class="alert alert-block alert-info">
<b>Info:</b> 
Here, the significant changes cloud is filtered manually to exclude fluvial erosion and deposition. The resulting cloud is given and will be used in the next steps.
</div>


#### Extract negative (erosion) and positive (deposits) distances 

<table><tr>
    <td> <img src="figures/Master_files_3.png" width=600 height=600/> </td>
</tr></table>

In [34]:
osf.open_folder(workspace+workflow_folder+segmentation_folder)

In [35]:
for i in range(0,len(Extraction_folder)):
    osf.open_folder(workspace+workflow_folder+segmentation_folder+list(Extraction_folder.values())[i])
    commande = cc.open_file(commande1,shift,workspace+workflow_folder+'significant_change_river_filtered.las')
    if i == 0:
        cc.filter_SF(commande,indexSF=0,min=-30,max=0)
        os.rename(workspace+workflow_folder+'significant_change_river_filtered_FILTERED_[-30_0].asc',workspace+workflow_folder+segmentation_folder+list(Extraction_folder.values())[i]+list(Extraction_filenames.values())[i]+'.asc')
    else:
        cc.filter_SF(commande,indexSF=0,min=0,max=30)
        os.rename(workspace+workflow_folder+'significant_change_river_filtered_FILTERED_[0_30].asc',workspace+workflow_folder+segmentation_folder+list(Extraction_folder.values())[i]+list(Extraction_filenames.values())[i]+'.asc')

### Volume estimation


<table><tr>
    <td> <img src="figures/Master_files_4.png" width=600 height=600/> </td>
</tr></table>

In [36]:
# Add "_3D" suffix to all columns 
fc.add_suffix_and_columns(workspace+workflow_folder+segmentation_folder,Extraction_folder,Extraction_filenames)

In [37]:
# Compute Vertical-M3C2
osf.open_folder(workspace+workflow_folder)
for i in range(0,len(Extraction_folder)):
    filenames['core_points'] = list(Extraction_filenames.values())[i]
    commande = cc.open_file(cc.open_file(cc.open_file(commande0,shift,workspace+Data_folder+filenames['pre_EQ_lidar']+'.laz'),shift,workspace+Data_folder+filenames['post_EQ_lidar']+'.laz'),shift,workspace+workflow_folder+segmentation_folder+list(Extraction_folder.values())[i]+filenames['core_points']+'.asc')
    cc.m3c2(commande,params_file_Vm3c2,workspace,Data_folder,filenames,'.las')
    os.rename(workspace+Data_folder+list(filenames.values())[0]+'_M3C2.las',workspace+workflow_folder+segmentation_folder+list(Extraction_folder.values())[i]+'Vertical_M3C2.las')

###  Segmentation by connected components


<table><tr>
    <td> <img src="figures/Master_files_5.png" width=800 height=800/> </td>
</tr></table>

In [3]:
commande2 = 'CloudCompare -SILENT -C_EXPORT_FMT BIN -AUTO_SAVE off -NO_TIMESTAMP '

In [4]:
for i in range(0,len(Extraction_folder)):
    osf.open_folder(workspace+workflow_folder+segmentation_folder+list(Extraction_folder.values())[i]+list(individual_outputs_folder.values())[i])
    # Create individual clouds and store the result in a .bin file
    commande = cc.open_file(commande2,shift,workspace+workflow_folder+segmentation_folder+list(Extraction_folder.values())[i]+'Vertical_M3C2.las')
    cc.connected_component(commande,octree_level,min_point_per_component)
    os.rename(workspace+workflow_folder+segmentation_folder+list(Extraction_folder.values())[i]+'AllClouds.bin',workspace+workflow_folder+segmentation_folder+list(Extraction_folder.values())[i]+list(individual_outputs_folder.values())[i]+list(individual_outputs_filenames.values())[i]+'.bin')
    # Merge all individual clouds
    commande = cc.open_file(commande0,shift,workspace+workflow_folder+segmentation_folder+list(Extraction_folder.values())[i]+list(individual_outputs_folder.values())[i]+list(individual_outputs_filenames.values())[i]+'.bin')
    cc.merge_clouds(commande)
    os.rename(workspace+workflow_folder+segmentation_folder+list(Extraction_folder.values())[i]+list(individual_outputs_folder.values())[i]+list(individual_outputs_filenames.values())[i]+'_MERGED_0.las',workspace+workflow_folder+segmentation_folder+list(Extraction_folder.values())[i]+list(individual_outputs_folder.values())[i]+list(individual_outputs_filenames.values())[i]+'_merged.las')

<div class="alert alert-block alert-warning">
<b>IMPORTANT:</b> In this study, the cloud number <b>#5497</b> in sources has been removed as it has been identified as resulting from a flight line imperfect alignement.
</div> 

<div class="alert alert-block alert-info">
<b>Tip:</b> 
Once done, you can then open the file "individual_sources.bin" or "individual_deposits.bin" in Cloudcompare. To save the information of each point cloud, select them and click on: *Tools > Batch export > Export cloud info*. .
</div>

In [12]:
raise SystemExit("The workflow stops right here!")

SystemExit: Stop right there!

# Results: Landslide analysis
<hr size="1"></hr>

## Geomorphic change and landslide detection inventory

In [20]:
path_clouds_infos = 'D:/Beyond_2D_inventories_synoptic_3D_landslide_volume_calculation_from_repeat_LiDAR_data/res/Supp._analysis/interpolation/'
filenames = {'Sources':'Sources_informations.csv','Deposits':'Deposits_informations.csv'}

In [21]:
fc.read_landslide_stats(path_clouds_infos,filenames)

,Number of sources/deposits,"Range of area (m² ; [min,max])",Total area (m²),Range of volume (m$^3$),Total volume (m$^3$),3D uncertainty on total volume
Sources,1431,"[20, 42650]",438124,"[6.97, 171175.0]",-908055.0,215640.0
Deposits,853,"[20, 32513]",376363,"[7.18, 154599.0]",1008626.0,172745.0
